# Финальный проект. Поиск

In [102]:
import requests
session = requests.session()
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
from html import unescape
from bs4 import BeautifulSoup
import re
from tqdm.auto import tqdm
import unicodedata
import sqlite3
import time
import random
import pandas as pd

## Подключаемся к базе данных

In [109]:
con = sqlite3.connect('project_nlp.db')

In [110]:
c = con.cursor()
con.commit()

## Функция, которая ищет точное вхождение словоформы

In [73]:
def search_token(qua):
    qua = qua.replace('"', '')
    x = (qua, )
    lemmas = []
    id_sents = []
    tokens = []
    pos = []
    id_tokens = []
    morph = []
    for row in c.execute("""SELECT id_token, id_sent, lemma, POS, morph FROM df_tokens
        WHERE token=?""", x):
        id_tokens.append(row[-5])
        id_sents.append(row[-4])
        lemmas.append(row[-3])
        pos.append(row[-2])
        morph.append(row[-1])
        
    sentences = []
    for row in c.execute("""SELECT sent FROM df_sents JOIN df_tokens
        ON df_sents.id_sent = df_tokens.id_sent WHERE token=?""", x):
        sentences.append(row[-1])
    
    links = []
    for row in c.execute("""SELECT link FROM df_texts JOIN df_tokens
        ON df_texts.id_text = df_tokens.id_text WHERE token=?""", x):
        links.append(row[-1])
    
    for i in range(len(id_sents)):
        tokens.append(qua)
        
    df_output = pd.DataFrame()
    df_output['id_token'] = id_tokens
    df_output['token'] = tokens
    df_output['lemma'] = lemmas
    df_output['POS'] = pos
    df_output['morph'] = morph
    df_output['sent'] = sentences
    df_output['link'] = links
    df_output['id'] = id_tokens
    df_output['id_sent'] = id_sents
    df_output.set_index('id', inplace=True)
    return df_output

## Функция, которая ищет по POS тегам

In [74]:
def search_pos(qua):
    x = (qua, )
    tokens = []
    lemmas = []
    id_sents = []
    tokens = []
    pos = []
    morph = []
    sentences = []
    id_tokens = []
    
    for row in c.execute("""SELECT id_token, id_sent, token, lemma, morph FROM df_tokens
        WHERE POS=?""", x):
        id_tokens.append(row[-5])
        id_sents.append(row[-4])
        lemmas.append(row[-3])
        tokens.append(row[-2])
        morph.append(row[-1])
        
    sentences = []
    for row in c.execute("""SELECT sent FROM df_sents JOIN df_tokens
        ON df_sents.id_sent = df_tokens.id_sent WHERE POS=?""", x):
        sentences.append(row[-1])
        
    links = []
    for row in c.execute("""SELECT link FROM df_texts JOIN df_tokens
        ON df_texts.id_text = df_tokens.id_text WHERE POS=?""", x):
        links.append(row[-1])
        
    for i in range(len(id_sents)):
        pos.append(qua)
        
    df_output = pd.DataFrame()
    df_output['id_token'] = id_tokens
    df_output['token'] = tokens
    df_output['lemma'] = lemmas
    df_output['POS'] = pos
    df_output['morph'] = morph
    df_output['sent'] = sentences
    df_output['link'] = links
    df_output['id'] = id_tokens
    df_output['id_sent'] = id_sents
    df_output.set_index('id', inplace=True)
    return df_output

## Функция, которая ищет все слово в любой форме

In [116]:
def search_all_forms(qua):
    lemmas_qua = lemmatization(qua)
    df_output = pd.DataFrame()
    otvet_token = []
    otvet_lemma = []
    otvet_pos = []
    otvet_morph = []
    otvet_sent = []
    otvet_link = []
    sent_ids = []
    text_ids = []
    id_tokens = []
    for lemma_qua in lemmas_qua:
        x = (lemma_qua, )
        for row in c.execute('''SELECT id_token, df_tokens.id_sent, df_tokens.id_text, sent, lemma, token, pos, morph FROM df_tokens
                      INNER JOIN df_sents ON df_tokens.id_sent = df_sents.id_sent
                      WHERE lemma=?''', x):
            id_tokens.append(row[-8])
            otvet_lemma.append(row[-4])
            otvet_token.append(row[-3])
            otvet_pos.append(row[-2])
            otvet_morph.append(row[-1])
            otvet_sent.append(row[-5])
            text_ids.append(row[-6])
            sent_ids.append(row[-7])
                        
    for text_id in text_ids:
        y = (text_id, )
        for row1 in c.execute('''SELECT df_texts.id_text, link FROM df_texts
                     WHERE df_texts.id_text=?''', y):
                                    otvet_link.append(row1[-1])
    df_output['id_token'] = id_tokens
    df_output['token'] = otvet_token
    df_output['lemma'] = otvet_lemma
    df_output['POS'] = otvet_pos
    df_output['morph'] = otvet_morph
    df_output['sent'] = otvet_sent
    df_output['link'] = otvet_link
    df_output['id'] = id_tokens
    df_output['id_sent'] = sent_ids
    df_output.set_index('id', inplace=True)
    return(df_output)

## Функция, которая ищет слово в определенной части речи

In [84]:
def search_token_plus_pos(qua):
    lemma_qua, pos_qua = qua.split('+')
    df_output = pd.DataFrame()
    otvet_token = []
    otvet_lemma = []
    otvet_pos = []
    otvet_morph = []
    otvet_sent = []
    otvet_link = []
    sent_ids = []
    text_ids = []
    id_tokens = []
    x = (lemma_qua, pos_qua,)
    for row in c.execute('''SELECT id_token, df_tokens.id_sent, df_tokens.id_text, sent, lemma, token, pos, morph FROM df_tokens
                INNER JOIN df_sents ON df_tokens.id_sent = df_sents.id_sent
                WHERE lemma=? AND pos=?''', x):
                        id_tokens.append(row[-8])
                        otvet_lemma.append(row[-4])
                        otvet_token.append(row[-3])
                        otvet_pos.append(row[-2])
                        otvet_morph.append(row[-1])
                        otvet_sent.append(row[-5])
                        text_ids.append(row[-6])
                        sent_ids.append(row[-7])
    for text_id in text_ids:
        y = (text_id, )
        for row1 in c.execute('''SELECT df_texts.id_text, link FROM df_texts
                     WHERE df_texts.id_text=?''', y):
                                    otvet_link.append(row1[-1])
    df_output['id_token'] = id_tokens
    df_output['token'] = otvet_token
    df_output['lemma'] = otvet_lemma
    df_output['POS'] = otvet_pos
    df_output['morph'] = otvet_morph
    df_output['sent'] = otvet_sent
    df_output['link'] = otvet_link
    df_output['id'] = id_tokens
    df_output['id_sent'] = sent_ids
    df_output.set_index('id', inplace=True)
    return(df_output)

## Лемматизация для поисковых запросов

Функция *lemmatization(token)* с помощью *morph_vocab.lemmatize()* из *Natasha* определяет все возможные леммы для поданной на вход словоформы и возвращает их список.

In [91]:
from natasha import MorphVocab

morph_vocab = MorphVocab()

def lemmatization(token):
    forms = morph_vocab(token)
    forms_lst = []
    for f in forms:
        forms_lst.append(list(f)[2])
    forms_lst = list(set(forms_lst))
    return forms_lst

## Определяем, какую функцию поиска нужно применить к введенному запросу

In [113]:
def detect_func(qua):
    pos_list = ['NOUN', 'VERB', 'PROPN', 'AUX', 'ADP', 'NUM', 'SYM', 'ADV', 'ADJ', 'CCONJ', 'SCONJ', 'DET', 'INYJ',
               'PART', 'PRON', 'PUNCT', 'X']
    if len(qua.split()) == 1:
        if '"' in qua:
            func = search_token(qua)
        elif '+' in qua:
            func = search_token_plus_pos(qua)
        elif qua in pos_list:
            func = search_pos(qua)
        elif qua.isalpha():
            func = search_all_forms(qua)
    return func

## Функция, которая выводит ответ для униграмм, биграмм и триграмм

In [88]:
def main(que):
    list_dfs = []
    split = que.split()
    if len(split) > 1:
        for x in split:
            func = detect_func(x)
            list_dfs.append(func)
    
    dicts = []
    for df in list_dfs:
        dict_df = {}
        dict_df = df.to_dict(orient='id')
        dicts.append(dict_df)
    
    ids_token1 = []
    ids_sent1 = []
    tokens1 = []
    ids_token2 = []
    ids_sent2 = []
    tokens2 = []
    lemma_1_list = []
    lemma_2_list = []
    pos_1_list = []
    pos_2_list = []
    morph_1_list = []
    morph_2_list = []
    ids_token3 = []
    ids_sent3 = []
    tokens3 = []
    lemma_3_list = []
    pos_3_list = []
    morph_3_list = []
    links = []
    sentences = []
    
    for key, value in dicts[0].items():
        k = key + 1
        if (k in dicts[1].keys()) and (value['id_sent'] == dicts[1][k]['id_sent']):   
            ids_token1.append(value['id_token'])
            ids_sent1.append(value['id_sent'])
            tokens1.append(value['token'])
            lemma_1_list.append(value['lemma'])
            pos_1_list.append(value['POS'])
            morph_1_list.append(value['morph'])
            sentences.append(value['sent'])
            links.append(value['link'])
            ids_token2.append(dicts[1][k]['id_token'])
            ids_sent2.append(dicts[1][k]['id_sent'])
            tokens2.append(dicts[1][k]['token'])
            lemma_2_list.append(dicts[1][k]['lemma'])
            pos_2_list.append(dicts[1][k]['POS'])
            morph_2_list.append(dicts[1][k]['morph'])
            
            if len(dicts) == 3:
                for key, value in dicts[1].items():
                    k = key + 1
                    if (k in dicts[2].keys()) and (value['id_sent'] == dicts[2][k]['id_sent']):
                        ids_token3.append(dicts[2][k]['id_token'])
                        ids_sent3.append(dicts[2][k]['id_sent'])
                        tokens3.append(dicts[2][k]['token'])
                        lemma_3_list.append(dicts[2][k]['lemma'])
                        pos_3_list.append(dicts[2][k]['POS'])
                        morph_3_list.append(dicts[2][k]['morph'])
    
    if len(sentences) == 0:
        print('В корпусе нет такого словосочетания')
    df_output = pd.DataFrame()
    df_output['sent'] = sentences
    df_output['token_1'] = tokens1
    df_output['lemma_1'] = lemma_1_list
    df_output['POS_1'] = pos_1_list
    df_output['morph_1'] = morph_1_list
    df_output['token_2'] = tokens2
    df_output['lemma_2'] = lemma_2_list
    df_output['POS_2'] = pos_2_list
    df_output['morph_2'] = morph_2_list
    if len(dicts) == 3:
        df_output['token_3'] = tokens3
        df_output['lemma_3'] = lemma_3_list
        df_output['POS_3'] = pos_3_list
        df_output['morph_3'] = morph_3_list
    df_output['link'] = links
    
                
    return df_output                

# Морфологическая разметка

https://universaldependencies.org/u/pos/

## Главная функция поиска

In [107]:
def search():
    qua = str(input('Введите запрос: '))
    if len(qua.split()) == 1:
        answer = detect_func(qua)
    if (len(qua.split()) == 2) or (len(qua.split()) == 3):
        answer = main(qua)
    if (len(qua.split()) == 0) or (len(qua.split()) > 3):
        print('Количество введёных запросов меньше 1 или больше 3 :(')
    return answer

In [111]:
search()

Введите запрос: NOUN знать+VERB


,sent,token_1,lemma_1,POS_1,morph_1,token_2,lemma_2,POS_2,morph_2,link
0,"Ибо понятно, что человек знающий, что его тран...",человек,человек,NOUN,"Animacy:Anim,Case:Nom,Gender:Masc,Number:Sing",знающий,знать,VERB,"Aspect:Imp,Case:Nom,Gender:Masc,Number:Sing,Te...",https://knife.media/mironov/
1,Если исследователь защищает диссертацию по пол...,тезис,тезисы,NOUN,"Animacy:Inan,Case:Acc,Gender:Masc,Number:Plur",зная,знать,VERB,"Aspect:Imp,Tense:Pres,VerbForm:Conv,Voice:Act",https://knife.media/mironov/
2,Эти первые дружеские отношения — тренировочная...,штаны,штаны,NOUN,"Animacy:Inan,Case:Acc,Gender:Masc,Number:Plur",знает,знать,VERB,"Aspect:Imp,Mood:Ind,Number:Sing,Person:3,Tense...",https://knife.media/what-is-your-name/
3,"/ Знает хороший ловец, где сети раскинуть на л...",лань,ланей,NOUN,"Animacy:Inan,Case:Loc,Gender:Masc,Number:Sing",Знает,знать,VERB,"Aspect:Imp,Mood:Ind,Number:Sing,Person:3,Tense...",https://knife.media/walks/
4,Фланера описывали и как привилегированного бур...,детектив,детектива,NOUN,"Animacy:Inan,Case:Gen,Gender:Masc,Number:Sing",знающего,знать,VERB,"Aspect:Perf,Case:Gen,Gender:Masc,Number:Sing,T...",https://knife.media/walks/
...,...,...,...,...,...,...,...,...,...,...
72,Право знать Четкого диагноза в нагромождении с...,право,Право,NOUN,"Animacy:Inan,Case:Nom,Gender:Neut,Number:Sing",знать,знать,VERB,"Aspect:Imp,VerbForm:Inf,Voice:Act",https://takiedela.ru/2019/02/vydressirovat-bol...
73,"Человек имеет право знать, что с ним происходи...",право,право,NOUN,"Animacy:Inan,Case:Acc,Gender:Neut,Number:Sing",знать,знать,VERB,"Aspect:Imp,VerbForm:Inf,Voice:Act",https://takiedela.ru/2019/02/vydressirovat-bol...
74,"О том, что во Владикавказе есть католический п...",приход,приход,NOUN,"Animacy:Inan,Case:Nom,Gender:Masc,Number:Sing",знают,знать,VERB,"Aspect:Imp,Mood:Ind,Number:Plur,Person:3,Tense...",https://takiedela.ru/2019/01/ekzotichesrkyi-ka...
75,Племянник знать никого не хочет из родственник...,племянник,Племянник,NOUN,"Animacy:Anim,Case:Nom,Gender:Masc,Number:Sing",знать,знать,VERB,"Aspect:Imp,VerbForm:Inf,Voice:Act",https://takiedela.ru/2018/11/khoroshego-vam-ra...


In [114]:
con.close()